# Serial Version and Spark Tuning (word-level correction)

In [ ]:
# word_level_documentation.ipynb

######################
#
# Submission by Kendrick Lo (Harvard ID: 70984997) for
# CS 205 - Computing Foundations for Computational Science (Prof. R. Jones)
# Advisor: A. Peleg
# 
# This is part of a joint project with Gioia Dominedo that includes a separate
# component for context checking. This notebook outlines algorithms for
# word-level correction, and includes a serial Python algorithm based on a third
# party algorithm (namely SymSpell, see below), as well as a Spark/Python
# algorithm. A number of optimizations/compromises were attempted with varying
# levels of success -- these attempts have been documented in this notebook.
#
######################

# contents

* [Performance Summary](#Performance-Summary) <p>

* [1a. word/document - original serial implementation](#1a.-serial-code-performance)
* [1b. single word - optimized serial version (SymSpell python port)](#1b.-optimized-serial-version)
* [2. single word - SPARK implementation (slow)](#2.-Original-SPARK-version:-SLOW)
* [3. single word - SPARK implementation (faster)](#3.-Optimized-SPARK-version:-FASTER)
* [4. single word - SPARK implementation (also fast)](#4.-Optimized-SPARK-version:-ALSO-FAST)
* [5. document - SPARK implementation](#5.-SPARK-version-document-check) <p>

* [AWS Experiments](#AWS-experiments)
* [Appendix: code snippets from optimization experiments](#Appendix)

# Performance Summary

<img src='img/times1.png'>

<img src='img/times2.png'>

### 1a. serial code performance

<div class="alert alert-info">
single word correction - corresponds to `serial_listsugg.py` <br>
document correction - corresponds to `serial_document.py`
</div>

In [ ]:
'''
v 1.0 last revised 22 Nov 2015

This program is a Python version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

Future improvements may entail allowing for less verbose options, 
including the output of a single recommended correction. Note also that we
have generally kept to the form of the original program, and have not
introduced any major optimizations or structural changes in this Python port.

To execute program:
1. Ensure "big.txt" is in the current working directory. This is the corpus
   from which the dictionary for the spellchecker will be built.
2a. Check recommended single word corrections by executing get_suggestions("word") 
    in the corresponding marked box below; or
2b. Check single word corrections for a document by executing
    correct_document("<file>") in the corresponding marked box below.

################

Example input/output:

################

get_suggestions("there")

number of possible corrections: 604
  edit distance for deletions: 3
  
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...

####

correct_document("OCRsample.txt")

Finding misspelled words in your document...
In line 3, taiths: suggested correction is < faith >
In line 11, the word < oonipiittee > was not found (no suggested correction)
In line 13, tj: suggested correction is < to >
In line 13, mnnff: suggested correction is < snuff >
[...]

total words checked: 700
total unknown words: 3
total potential errors found: 94

'''

import re

max_edit_distance = 3 

dictionary = {}
longest_word_length = 0

def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added

def create_dictionary(fname):

    total_word_count = 0
    unique_word_count = 0
    print "Creating dictionary..." 
    
    with open(fname) as file:    
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                total_word_count += 1
                if create_dictionary_entry(word):
                    unique_word_count += 1
    
    print "total words processed: %i" % total_word_count
    print "total unique words in corpus: %i" % unique_word_count
    print "total items in dictionary (corpus words and deletions): %i" % len(dictionary)
    print "  edit distance for deletions: %i" % max_edit_distance
    print "  length of longest word in corpus: %i" % longest_word_length
        
    return dictionary

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    This method has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_suggestions(string, silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))
            
            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
             
    # queue is now empty: convert suggestions in dictionary to list for output
    if not silent:
        print "number of possible corrections: %i" %len(suggest_dict)
        print "  edit distance for deletions: %i" % max_edit_distance
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested word corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    return sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

def best_word(s, silent=False):
    try:
        return get_suggestions(s, silent)[0]
    except:
        return None
    
def correct_document(fname, printlist=True):
    # correct an entire document
    with open(fname) as file:
        doc_word_count = 0
        corrected_word_count = 0
        unknown_word_count = 0
        print "Finding misspelled words in your document..." 
        
        for i, line in enumerate(file):
            doc_words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words:
                doc_word_count += 1
                suggestion = best_word(doc_word, silent=True)
                if suggestion is None:
                    if printlist:
                        print "In line %i, the word < %s > was not found (no suggested correction)" % (i, doc_word)
                    unknown_word_count += 1
                elif suggestion[0]!=doc_word:
                    if printlist:
                        print "In line %i, %s: suggested correction is < %s >" % (i, doc_word, suggestion[0])
                    corrected_word_count += 1
        
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count

    return

<div class="alert alert-danger">
  <strong>Run the cell below only once to build the dictionary.</strong>
</div>

In [ ]:
d = create_dictionary("testdata/big.txt")

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 26.2 s, sys: 556 ms, total: 26.8 s
Wall time: 26.8 s
</code></pre>

<div class="alert alert-success">
  <strong>Enter word to correct below.</strong>
</div>

In [ ]:
%%time
get_suggestions("there")

<pre><code>
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 60.6 ms, sys: 11.5 ms, total: 72.1 ms
Wall time: 64 ms
Out[8]:
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...
</code></pre>

In [ ]:
%%time
get_suggestions("zzffttt")

<pre><code>
number of possible corrections: 0
  edit distance for deletions: 3
CPU times: user 191 µs, sys: 81 µs, total: 272 µs
Wall time: 208 µs
Out[4]:
[]
</code></pre>

In [ ]:
%%time
best_word("there")

<pre><code>
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 55.3 ms, sys: 2.97 ms, total: 58.2 ms
Wall time: 57.2 ms
Out[4]:
('there', (2972, 0))
</code></pre>

<div class="alert alert-success">
  <strong>Enter file name of document to correct below.</strong>
</div>

In [ ]:
%%time
correct_document("testdata/OCRsample.txt")

<pre><code>
Finding misspelled words in your document...
In line 3, taiths: suggested correction is < faith >
In line 11, the word < oonipiittee > was not found (no suggested correction)
In line 13, tj: suggested correction is < to >
In line 13, mnnff: suggested correction is < snuff >
In line 13, gjpt: suggested correction is < get >
In line 15, bh: suggested correction is < by >
In line 15, snc: suggested correction is < sac >
In line 15, uth: suggested correction is < th >
In line 15, unuer: suggested correction is < under >
In line 20, mthiitt: suggested correction is < thirty >
In line 21, cas: suggested correction is < was >
In line 22, pythian: suggested correction is < scythian >
In line 26, brainin: suggested correction is < brain >
In line 27, jfl: suggested correction is < of >
In line 28, ji: suggested correction is < i >
In line 28, stice: suggested correction is < stick >
In line 28, blaci: suggested correction is < black >
In line 28, eug: suggested correction is < dug >
In line 28, debbs: suggested correction is < debts >
In line 29, nericans: suggested correction is < americans >
In line 30, ainin: suggested correction is < again >
In line 30, ergs: suggested correction is < eggs >
In line 31, trumped: suggested correction is < trumpet >
In line 32, erican: suggested correction is < american >
In line 33, unorthodox: suggested correction is < orthodox >
In line 33, nenance: suggested correction is < penance >
In line 33, thg: suggested correction is < the >
In line 34, sln: suggested correction is < son >
In line 34, rgs: suggested correction is < rags >
In line 38, williaij: suggested correction is < william >
In line 38, eu: suggested correction is < e >
In line 40, fcsf: suggested correction is < ff >
In line 40, ber: suggested correction is < be >
In line 42, unorthodoxy: suggested correction is < orthodox >
In line 42, thpt: suggested correction is < that >
In line 42, the word < senbrnrgs > was not found (no suggested correction)
In line 44, fascism: suggested correction is < fascia >
In line 62, loo: suggested correction is < look >
In line 65, ththn: suggested correction is < then >
In line 65, scbell: suggested correction is < bell >
In line 65, ife: suggested correction is < if >
In line 65, yktcn: suggested correction is < skin >
In line 65, thl: suggested correction is < the >
In line 66, thi: suggested correction is < the >
In line 68, saij: suggested correction is < said >
In line 69, defendants: suggested correction is < defendant >
In line 69, cornr: suggested correction is < corner >
In line 69, nists: suggested correction is < fists >
In line 72, ro: suggested correction is < to >
In line 74, ath: suggested correction is < at >
In line 75, tti: suggested correction is < ti >
In line 75, rg: suggested correction is < re >
In line 75, acrific: suggested correction is < pacific >
In line 77, korea: suggested correction is < more >
In line 78, ro: suggested correction is < to >
In line 78, doatli: suggested correction is < death >
In line 81, ith: suggested correction is < it >
In line 81, ry: suggested correction is < by >
In line 81, kl: suggested correction is < ll >
In line 81, ech: suggested correction is < each >
In line 82, rb: suggested correction is < re >
In line 82, the word < ghmhvestigat > was not found (no suggested correction)
In line 82, nb: suggested correction is < no >
In line 82, rg: suggested correction is < re >
In line 83, rosenbt: suggested correction is < rodent >
In line 83, rgs: suggested correction is < rags >
In line 84, coriritted: suggested correction is < committed >
In line 86, fighti: suggested correction is < fight >
In line 88, bths: suggested correction is < baths >
In line 88, tchf: suggested correction is < the >
In line 91, ro: suggested correction is < to >
In line 91, ijb: suggested correction is < in >
In line 92, telegrnm: suggested correction is < telegram >
In line 92, jillia: suggested correction is < william >
In line 92, patt: suggested correction is < part >
In line 92, rson: suggested correction is < son >
In line 93, ecretdry: suggested correction is < secretary >
In line 95, purview: suggested correction is < purves >
In line 95, rder: suggested correction is < order >
In line 99, gor: suggested correction is < for >
In line 99, dthethg: suggested correction is < teeth >
In line 99, ared: suggested correction is < are >
In line 99, ro: suggested correction is < to >
In line 99, enb: suggested correction is < end >
In line 99, rg: suggested correction is < re >
In line 100, sacc: suggested correction is < sac >
In line 100, vthnz: suggested correction is < the >
In line 100, dri: suggested correction is < dry >
In line 100, yfu: suggested correction is < you >
In line 101, ile: suggested correction is < ill >
In line 101, rosi: suggested correction is < rose >
In line 101, rg: suggested correction is < re >
In line 102, fnir: suggested correction is < fair >
In line 102, jhy: suggested correction is < why >
In line 102, azi: suggested correction is < ami >
In line 103, fascist: suggested correction is < fascia >
In line 104, nb: suggested correction is < no >
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 14.5 s, sys: 147 ms, total: 14.6 s
Wall time: 15 s
</code></pre>

### 1b. optimized serial version

<div class="alert alert-info">
The above version did not incorporate all of the functionality of the original SymSpell code. For example, the above version did not terminate early if the user only wanted to find a single, best word. It also did not allow for interactive input (e.g. from the console). We added these features to the final version below for completeness. <br><p>

corresponds to `serial_single.py`
</div>

In [ ]:
'''
v 1.1 last revised 28 Nov 2015

This program is a Python version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

Changes in this version (1.1):
We implement allowing for less verbose options: e.g. when only a single recommended
correction is required, the search may terminate early, thereby enhancing performance. 

Sample output:

Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
 
Word correction
---------------
Enter your input (or enter to exit): there
('there', (2972, 0))
 
Enter your input (or enter to exit): hellot
('hello', (1, 1))
 
Enter your input (or enter to exit): accomodation
('accommodation', (5, 1))
 
Enter your input (or enter to exit): 
goodbye

'''

import re

max_edit_distance = 3 
verbose = 0
# 0: top suggestion
# 1: all suggestions of smallest edit distance
# 2: all suggestions <= max_edit_distance (slower, no early termination)

dictionary = {}
longest_word_length = 0

def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added

def create_dictionary(fname):

    total_word_count = 0
    unique_word_count = 0
    print "Creating dictionary..." 
    
    with open(fname) as file:    
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                total_word_count += 1
                if create_dictionary_entry(word):
                    unique_word_count += 1
    
    print "total words processed: %i" % total_word_count
    print "total unique words in corpus: %i" % unique_word_count
    print "total items in dictionary (corpus words and deletions): %i" % len(dictionary)
    print "  edit distance for deletions: %i" % max_edit_distance
    print "  length of longest word in corpus: %i" % longest_word_length
        
    return dictionary

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    This method has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_suggestions(string, silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []
    
    global verbose
    suggest_dict = {}
    min_suggest_len = float('inf')
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # early exit
        if ((verbose<2) and (len(suggest_dict)>0) and ((len(string)-len(q_item))>min_suggest_len)):
            break
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))
                # early exit
                if ((verbose<2) and (len(string)==len(q_item))):
                    break
                elif (len(string) - len(q_item)) < min_suggest_len:
                    min_suggest_len = len(string) - len(q_item)
            
            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    # do not add words with greater edit distance if verbose setting not on
                    if ((verbose<2) and (item_dist>min_suggest_len)):
                        pass
                    elif item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
                        if item_dist < min_suggest_len:
                            min_suggest_len = item_dist
                    
                    # depending on order words are processed, some words with different edit distances
                    # may be entered into suggestions; trim suggestion dictionary if verbose setting not on
                    if verbose<2:
                        suggest_dict = {k:v for k, v in suggest_dict.items() if v[1]<=min_suggest_len}
                
        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
                    
        # do not add words with greater edit distance if verbose setting not on
        if ((verbose<2) and ((len(string)-len(q_item))>min_suggest_len)):
            pass
        elif (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
                     
    # queue is now empty: convert suggestions in dictionary to list for output
    if not silent and verbose!=0:
        print "number of possible corrections: %i" %len(suggest_dict)
        print "  edit distance for deletions: %i" % max_edit_distance
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested word corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    outlist = sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))
    
    if verbose==0:
        return outlist[0]
    else:
        return outlist

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

def best_word(s, silent=False):
    try:
        return get_suggestions(s, silent)[0]
    except:
        return None
    
def correct_document(fname, printlist=True):
    # correct an entire document
    with open(fname) as file:
        doc_word_count = 0
        corrected_word_count = 0
        unknown_word_count = 0
        print "Finding misspelled words in your document..." 
        
        for i, line in enumerate(file):
            doc_words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words:
                doc_word_count += 1
                suggestion = best_word(doc_word, silent=True)
                if suggestion is None:
                    if printlist:
                        print "In line %i, the word < %s > was not found (no suggested correction)" % (i, doc_word)
                    unknown_word_count += 1
                elif suggestion[0]!=doc_word:
                    if printlist:
                        print "In line %i, %s: suggested correction is < %s >" % (i, doc_word, suggestion[0])
                    corrected_word_count += 1
        
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count

    return

## main

import time

if __name__ == "__main__":
    
    print "Please wait..."
    time.sleep(2)
    create_dictionary("testdata/big.txt")

    print " "
    print "Word correction"
    print "---------------"
    
    while True:
        word_in = raw_input('Enter your input (or enter to exit): ')
        if len(word_in)==0:
            print "goodbye"
            break
        print get_suggestions(word_in)
        print " "

<div class="alert alert-success">
  <strong>Simply run the code block above and follow the interactive instructions.</strong>
</div>

<pre><code>
Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
 
Word correction
---------------
Enter your input (or enter to exit): hello
('hello', (1, 0))
 
Enter your input (or enter to exit): there
('there', (2972, 0))
 
Enter your input (or enter to exit): thinkl
('think', (557, 1))
 
Enter your input (or enter to exit): prest
('rest', (209, 1))
 
Enter your input (or enter to exit): 
goodbye
</code></pre>

------

### 2. Original SPARK version: SLOW

<div class="alert alert-danger">
  <strong>To run this program, restart notebook, and start executing the cells of this section starting here.</strong> <p>
  WARNING: This version is extremely slow for even moderately sized files.
</div>

<div class="alert alert-info">
corresponds to `spark_1.py`
</div>

In [ ]:
'''
v 1.0 last revised 22 Nov 2015

This program is a Spark (PySpark) version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

Future improvements may entail allowing for less verbose options, 
including the output of a single recommended correction. Note also that we
have generally kept to the form of the original program, and have not
introduced any major optimizations or structural changes in this PySpark port.

To execute program:
1. Ensure "big.txt" is in the current working directory. This is the corpus
   from which the dictionary for the spellchecker will be built.
2. Check recommended single word corrections by executing get_suggestions("word") 
    in the corresponding marked box below.

Note: we did not implement entire document checking given speed of program,
      since we are parallelizing the processing of deletions of the input word
      (however, see later Spark version).

################

Example input/output:

################

get_suggestions("there")

number of possible corrections: 604
  edit distance for deletions: 3
  
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...


'''

import findspark
import os
findspark.init('/Users/K-Lo/spark-1.5.0')

import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

import re

n_partitions = 6  # number of partitions to be used
max_edit_distance = 3


def get_deletes_list(word):
    '''given a word, derive strings with one character deleted'''
    # takes a string as input and returns all 1-deletes in a list
    # allows for duplicates to be created, will deal with duplicates later to minimize shuffling
    if len(word)>1:
        return ([word[:c] + word[c+1:] for c in range(len(word))])
    else:
        return []
    
def copartitioned(RDD1, RDD2):
    '''check if two RDDs are copartitioned'''
    return RDD1.partitioner == RDD2.partitioner

def combine_joined_lists(tup):
    '''takes as input a tuple in the form (a, b) where each of a, b may be None (but not both) or a list
       and returns a concatenated list of unique elements'''
    concat_list = []
    if tup[1] is None:
        concat_list = tup[0]
    elif tup[0] is None:
        concat_list = tup[1]
    else:
        concat_list = tup[0] + tup[1]
        
    return list(set(concat_list))

def parallel_create_dictionary(fname):
    '''Create dictionary using Spark RDDs.'''
    # we generate and count all words for the corpus,
    # then add deletes to the dictionary
    # this is a slightly different approach from the SymSpell algorithm
    # that may be more appropriate for Spark processing
    
    print "Creating dictionary..." 
    
    ############
    #
    # process corpus
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # convert file into one long sequence of words
    make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
    replace_nonalphs = make_all_lower.map(lambda line: regex.sub(' ', line))
    all_words = replace_nonalphs.flatMap(lambda line: line.split())

    # create core corpus dictionary (i.e. only words appearing in file, no "deletes") and cache it
    # output RDD of unique_words_with_count: [(word1, count1), (word2, count2), (word3, count3)...]
    count_once = all_words.map(lambda word: (word, 1))
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()
    
    # output stats on core corpus
    print "total words processed: %i" % unique_words_with_count.map(lambda (k, v): v).reduce(lambda a, b: a + b)
    print "total unique words in corpus: %i" % unique_words_with_count.count()
    
    ############
    #
    # generate deletes list
    #
    ############
    
    # generate list of n-deletes from words in a corpus of the form: [(word1, count1), (word2, count2), ...]
    # we will handle possible duplicates after map/reduce:
    #     our thinking is the resulting suggestions lists for each delete will be much smaller than the
    #     list of potential deletes, and it is more efficient to reduce first, then remove duplicates 
    #     from these smaller lists (at each worker node), rather than calling `distinct()` on  
    #     flattened `expand_deletes` which would require a large shuffle

    ##
    ## generate 1-deletes
    ##
     
    assert max_edit_distance>0  
    
    generate_deletes = unique_words_with_count.map(lambda (parent, count): (parent, get_deletes_list(parent)), 
                                                      preservesPartitioning=True)
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    
    # swap and combine, resulting RDD after processing 1-deletes has elements:
    # [(delete1, [correct1, correct2...]), (delete2, [correct1, correct2...])...]
    swap = expand_deletes.map(lambda (orig, delete): (delete, [orig]))
    combine = swap.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions)

    # cache "master" deletes RDD, list of (deletes, [unique suggestions]), for use in loop
    deletes = combine.mapValues(lambda sl: list(set(sl))).cache()
    
    ##
    ## generate 2+ deletes
    ##
    
    d_remaining = max_edit_distance - 1  # decreasing counter
    queue = deletes

    while d_remaining>0:

        # generate further deletes -- we parallelize processing of all deletes in this version
        #'expand_new_deletes' will be of the form [(parent "delete", [new child "deletes"]), ...]
        # n.b. this will filter out elements with no new child deletes
        gen_new_deletes = queue.map(lambda (x, y): (x, get_deletes_list(x)), preservesPartitioning=True)
        expand_new_deletes = gen_new_deletes.flatMapValues(lambda x: x)  

        # associate each new child delete with same corpus word suggestions that applied for parent delete
        # update queue with [(new child delete, [corpus suggestions]) ...] and cache for next iteration
        
        assert copartitioned(queue, expand_new_deletes)   # check partitioning for efficient join
        get_sugglist_from_parent = expand_new_deletes.join(queue)
        new_deletes = get_sugglist_from_parent.map(lambda (p, (c, sl)): (c, sl))
        combine_new = new_deletes.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions)
        queue = combine_new.mapValues(lambda sl: list(set(sl))).cache()

        # update "master" deletes list with new deletes, and cache for next iteration
        
        assert copartitioned(deletes, queue)    # check partitioning for efficient join
        join_delete_lists = deletes.fullOuterJoin(queue)
        deletes = join_delete_lists.mapValues(lambda y: combine_joined_lists(y)).cache()

        d_remaining -= 1
        
    ############
    #
    # merge deletes with unique corpus words to construct main dictionary
    #
    ############

    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    # note frequency of word in corpus is not incremented for deletes
    deletes_for_dict = deletes.mapValues(lambda sl: (sl, 0)) 
    unique_words_for_dict = unique_words_with_count.mapValues(lambda count: ([], count))

    assert copartitioned(unique_words_for_dict, deletes_for_dict)  # check partitioning for efficient join
    join_deletes = unique_words_for_dict.fullOuterJoin(deletes_for_dict)
    '''
    entries now in form of (word, ( ([], count), ([suggestions], 0) )) for words in both corpus/deletes
                           (word, ( ([], count), None               )) for (real) words in corpus only
                           (word, ( None       , ([suggestions], 0) )) for (fake) words in deletes only
    '''

    # if entry has deletes and is a real word, take suggestion list from deletes and count from corpus
    dictionary_RDD = join_deletes.mapValues(lambda (xtup, ytup): 
                                                xtup if ytup is None
                                                else ytup if xtup is None
                                                else (ytup[0], xtup[1])).cache()

    print "total items in dictionary (corpus words and deletions): %i" % dictionary_RDD.count()
    print "  edit distance for deletions: %i" % max_edit_distance
    longest_word_length = unique_words_with_count.map(lambda (k, v): len(k)).reduce(max)
    print "  length of longest word in corpus: %i" % longest_word_length
        
    return dictionary_RDD, longest_word_length

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance (an integer) between sequences.

    This code has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_n_deletes_list(w, n):
    '''given a word, derive strings with up to n characters deleted'''
    deletes = []
    queue = [w]
    for d in range(n):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def get_suggestions(s, dictRDD, longest_word_length=float('inf'), silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word.
    
    s: input string
    dictRDD: the main dictionary, which includes deletes
             entries are in the form of: [(word, ([suggested corrections], frequency of word in corpus)), ...]
    longest_word_length: optional identifier of longest real word in dictRDD
    silent: verbose output
    '''

    if (len(s) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []

    ##########
    #
    # initialize suggestions RDD
    # suggestRDD entries: (word, (frequency of word in corpus, edit distance))
    #
    ##########
    
    if not silent:
        print "looking up suggestions based on input word..."
    
    # ensure input RDDs are partitioned
    dictRDD = dictRDD.partitionBy(n_partitions).cache()
    
    # check if input word is in dictionary, and is a word from the corpus (edit distance = 0)
    # if so, add input word itself to suggestRDD
    exact_match = dictRDD.filter(lambda (w, (sl, freq)): w==s).cache()
    suggestRDD = exact_match.mapValues(lambda (sl, freq): (freq, 0)).cache()

    ##########
    #
    # add suggestions for input word
    #
    ##########
    
    # the suggested corrections for the item in dictionary (whether or not
    # the input string s itself is a valid word or merely a delete) can be valid corrections
    sc_items = exact_match.flatMap(lambda (w, (sl, freq)): sl)
    calc_dist = sc_items.map(lambda sc: (sc, len(sc)-len(s))).partitionBy(n_partitions).cache()
    
    assert copartitioned(dictRDD, calc_dist)  # check partitioning for efficient join
    get_freq = dictRDD.join(calc_dist)
    parent_sugg = get_freq.mapValues(lambda ((sl, freq), dist): (freq, dist))
    suggestRDD = suggestRDD.union(parent_sugg).cache()
    assert copartitioned(parent_sugg, suggestRDD)  # check partitioning

    ##########
    #
    # process deletes on the input string
    #
    ##########
     
    assert max_edit_distance>0
    
    list_deletes_of_s = sc.parallelize(get_n_deletes_list(s, max_edit_distance))
    deletes_of_s = list_deletes_of_s.map(lambda k: (k, 0)).partitionBy(n_partitions).cache()
    
    assert copartitioned(dictRDD, deletes_of_s) # check partitioning for efficient join
    check_matches = dictRDD.join(deletes_of_s).cache()
    
    # if delete is a real word in corpus, add it to suggestion list
    del_exact_match = check_matches.filter(lambda (w, ((sl, freq), _)): freq>0)
    del_sugg = del_exact_match.map(lambda (w, ((s1, freq), _)): (w, (freq, len(s)-len(w))),
                                   preservesPartitioning=True)
    suggestRDD = suggestRDD.union(del_sugg).cache()
    
    # the suggested corrections for the item in dictionary (whether or not
    # the delete itself is a valid word or merely a delete) can be valid corrections    
    list_sl = check_matches.mapValues(lambda ((sl, freq), _): sl).flatMapValues(lambda x: x)
    swap_del = list_sl.map(lambda (w, sc): (sc, 0))
    combine_del = swap_del.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()

    # need to recalculate actual Deverau-Levenshtein distance to be within max_edit_distance for all deletes
    calc_dist = combine_del.map(lambda (w, _): (w, dameraulevenshtein(s, w)),
                                       preservesPartitioning=True)
    filter_by_dist = calc_dist.filter(lambda (w, dist): dist<=max_edit_distance)
    
    # get frequencies from main dictionary and add to suggestions list
    assert copartitioned(dictRDD, filter_by_dist)  # check partitioning for efficient join
    get_freq = dictRDD.join(filter_by_dist)
    del_parent_sugg = get_freq.mapValues(lambda ((sl, freq), dist): (freq, dist))
    
    suggestRDD = suggestRDD.union(del_parent_sugg).distinct().cache()    
    
    if not silent:
        print "number of possible corrections: %i" %suggestRDD.count()
        print "  edit distance for deletions: %i" % max_edit_distance

    ##########
    #
    # sort RDD for output
    #
    ##########
    
    # suggest_RDD is in the form: [(word, (freq, editdist)), (word, (freq, editdist)), ...]
    # there does not seem to be a straightforward way to sort by both primary and secondary keys in Spark
    # this is a documented issue: one option is to simply work with a list since there are likely not
    # going to be an extremely large number of recommended suggestions
    
    output = suggestRDD.collect()
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(output, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    # return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))

    return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))

def best_word(s, d, l, silent=False):
    a = get_suggestions(s, d, l, silent)
    if len(a)==0:
        return (None, (None, None))
    else: 
        return a[0]

<div class="alert alert-danger">
  <strong>Run the cell below only once to build the dictionary.</strong>
</div>

In [ ]:
%%time
d, lwl = parallel_create_dictionary("testdata/big.txt")

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 113 ms, sys: 29.4 ms, total: 142 ms
Wall time: 5min 26s
</code></pre>

<div class="alert alert-success">
  <strong>Enter word to correct below.</strong>
</div>

In [ ]:
%%time
get_suggestions("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 79.9 ms, sys: 20.2 ms, total: 100 ms
Wall time: 1min 33s
Out[6]:
[(u'there', (2972, 0)),
 (u'these', (1231, 1)),
 (u'where', (977, 1)),
 (u'here', (691, 1)),
 (u'three', (584, 1)),
 (u'thee', (26, 1)),
 (u'chere', (9, 1)),
 (u'theme', (8, 1)), ...
</code></pre>

In [ ]:
%%time
get_suggestions("zzffttt", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 0
  edit distance for deletions: 3
CPU times: user 76.2 ms, sys: 18.9 ms, total: 95.1 ms
Wall time: 2min 7s
Out[5]:
[]
</code></pre>

In [ ]:
%%time
best_word("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 95.8 ms, sys: 22.7 ms, total: 119 ms
Wall time: 1min 33s
Out[4]:
(u'there', (2972, 0))
</code></pre>

----

### 3. Optimized SPARK version: FASTER

<div class="alert alert-danger">
  <strong>To run this program, restart notebook, and start executing the cells of this section starting here.</strong>
</div>

<div class="alert alert-info">
corresponds to `spark_2.py`
</div>

In [ ]:
'''
v 2.0 last revised 26 Nov 2015

This program is a Spark (PySpark) version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

Future improvements may entail allowing for less verbose options, 
including the output of a single recommended correction. Note also that we
have generally kept to the form of the original program, and have not
introduced any major optimizations or structural changes in this PySpark port.

To execute program:
1. Ensure "big.txt" is in the current working directory. This is the corpus
   from which the dictionary for the spellchecker will be built.
2. Check recommended single word corrections by executing get_suggestions("word") 
    in the corresponding marked box below.

Note: we did not implement entire document checking given speed of program,
      since we are parallelizing the processing of deletions of the input word
      (however, see later Spark version).

################

Example input/output:

################

get_suggestions("there")

number of possible corrections: 604
  edit distance for deletions: 3
  
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...


'''

import findspark
import os
findspark.init('/Users/K-Lo/spark-1.5.0')

from pyspark import SparkContext
sc = SparkContext()

import re

n_partitions = 6  # number of partitions to be used
max_edit_distance = 3

# helper functions
def get_n_deletes_list(w, n):
    '''given a word, derive list of strings with up to n characters deleted'''
    # since this list is generally of the same magnitude as the number of 
    # characters in a word, it may not make sense to parallelize this
    # so we use python to create the list
    deletes = []
    queue = [w]
    for d in range(n):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes
    
def copartitioned(RDD1, RDD2):
    '''check if two RDDs are copartitioned'''
    return RDD1.partitioner == RDD2.partitioner

def combine_joined_lists(tup):
    '''takes as input a tuple in the form (a, b) where each of a, b may be None (but not both) or a list
       and returns a concatenated list of unique elements'''
    concat_list = []
    if tup[1] is None:
        concat_list = tup[0]
    elif tup[0] is None:
        concat_list = tup[1]
    else:
        concat_list = tup[0] + tup[1]
        
    return list(set(concat_list))

def parallel_create_dictionary(fname):
    '''Create dictionary using Spark RDDs.'''
    # we generate and count all words for the corpus,
    # then add deletes to the dictionary
    # this is a slightly different approach from the SymSpell algorithm
    # that may be more appropriate for Spark processing
    
    print "Creating dictionary..." 
    
    ############
    #
    # process corpus
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # convert file into one long sequence of words
    make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
    replace_nonalphs = make_all_lower.map(lambda line: regex.sub(' ', line))
    all_words = replace_nonalphs.flatMap(lambda line: line.split())

    # create core corpus dictionary (i.e. only words appearing in file, no "deletes") and cache it
    # output RDD of unique_words_with_count: [(word1, count1), (word2, count2), (word3, count3)...]
    count_once = all_words.map(lambda word: (word, 1))
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()
    
    # output stats on core corpus
    print "total words processed: %i" % unique_words_with_count.map(lambda (k, v): v).reduce(lambda a, b: a + b)
    print "total unique words in corpus: %i" % unique_words_with_count.count()
    
    ############
    #
    # generate deletes list
    #
    ############
    
    # generate list of n-deletes from words in a corpus of the form: [(word1, count1), (word2, count2), ...]
     
    assert max_edit_distance>0  
    
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
                                                   (parent, get_n_deletes_list(parent, max_edit_distance)))
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
   
    ############
    #
    # combine delete elements with main dictionary
    #
    ############
    
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))
    combine = swap.union(corpus)  # combine deletes with main dictionary, eliminate duplicates
    new_dict = combine.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])).cache()
    
    print "total items in dictionary (corpus words and deletions): %i" % new_dict.count()
    print "  edit distance for deletions: %i" % max_edit_distance
    longest_word_length = unique_words_with_count.map(lambda (k, v): len(k)).reduce(max)
    print "  length of longest word in corpus: %i" % longest_word_length

    return new_dict, longest_word_length  

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance (an integer) between sequences.

    This code has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_suggestions(s, dictRDD, longest_word_length=float('inf'), silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word.
    
    s: input string
    dictRDD: the main dictionary, which includes deletes
             entries are in the form of: [(word, ([suggested corrections], frequency of word in corpus)), ...]
    longest_word_length: optional identifier of longest real word in dictRDD
    silent: verbose output
    '''

    if (len(s) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []

    ##########
    #
    # initialize suggestions RDD
    # suggestRDD entries: (word, (frequency of word in corpus, edit distance))
    #
    ##########
    
    if not silent:
        print "looking up suggestions based on input word..."
    
    # ensure input RDDs are partitioned
    dictRDD = dictRDD.repartitionAndSortWithinPartitions(n_partitions).cache()
    
    # check if input word is in dictionary, and is a word from the corpus (edit distance = 0)
    # if so, add input word itself to suggestRDD
    exact_match = dictRDD.filter(lambda (w, (sl, freq)): w==s).cache()
    suggestRDD = exact_match.mapValues(lambda (sl, freq): (freq, 0))
    
    ##########
    #
    # add suggestions for input word
    #
    ##########

    # the suggested corrections for the item in dictionary (whether or not
    # the input string s itself is a valid word or merely a delete) can be valid corrections
    # the suggestions list will likely be short: it is only for one word
    # so we choose to collect here and process as a list, rather than parallelizing 
    # a very short list
    sc_items = exact_match.flatMap(lambda (w, (sl, freq)): sl).collect()  
    get_freq = dictRDD.filter(lambda (w, (sl, freq)): w in sc_items)
    parent_sugg = get_freq.map(lambda (w, (sl, freq)): (w, (freq, len(w)-len(s))), 
                                   preservesPartitioning=True)
    suggestRDD = suggestRDD.union(parent_sugg).cache()
    assert copartitioned(parent_sugg, suggestRDD)  # check partitioning

    ##########
    #
    # process deletes of the input string
    #
    ##########
     
    assert max_edit_distance>0
    
    list_deletes_of_s = get_n_deletes_list(s, max_edit_distance)  # this list is also short
    check_matches = dictRDD.filter(lambda (w, (sl, freq)): w in list_deletes_of_s).cache()

    # identify deletes that match a dictionary entry, and add matches to suggestions
    del_exact_match = check_matches.filter(lambda (w, (sl, freq)): freq>0)
    del_sugg = del_exact_match.map(lambda (w, (s1, freq)): (w, (freq, len(s)-len(w))),
                                   preservesPartitioning=True)
    suggestRDD = suggestRDD.union(del_sugg).cache()
    
    ##########
    #
    # now process suggestions lists of deletes
    #
    ##########

    # the suggested corrections for the item in dictionary (whether or not
    # the delete itself is a valid word or merely a delete) can be valid corrections 
    list_sl = check_matches.mapValues(lambda (sl, freq): sl).flatMapValues(lambda x: x)
    swap_del = list_sl.map(lambda (w, sc): (sc, 0))
    combine_del = swap_del.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()

    # need to recalculate actual Deverau Levenshtein distance to be within max_edit_distance 
    # for all deletes and check against the threshold value
    calc_dist = combine_del.map(lambda (w, _): (w, dameraulevenshtein(s, w)),
                                       preservesPartitioning=True)
    filter_by_dist = calc_dist.filter(lambda (w, dist): dist<=max_edit_distance)
    
    # MERGE: get frequencies from main dictionary and add to suggestions list
    assert copartitioned(dictRDD, filter_by_dist)  # check partitioning for efficient join
    get_freq = dictRDD.join(filter_by_dist)
    del_parent_sugg = get_freq.mapValues(lambda ((sl, freq), dist): (freq, dist)).cache()
    
    suggestRDD = suggestRDD.union(del_parent_sugg).distinct().cache()    

    ##########
    #
    # output suggestions
    #
    ##########
    
    if not silent:
        print "number of possible corrections: %i" % suggestRDD.count()
        print "  edit distance for deletions: %i" % max_edit_distance

    output = suggestRDD.collect()
    
    # suggest_RDD is in the form: [(word, (freq, editdist)), (word, (freq, editdist)), ...]
    # there does not seem to be a straightforward way to sort by both primary and secondary keys in Spark
    # this is a documented issue: one option is to simply work with a list since there are likely not
    # going to be an extremely large number of recommended suggestions
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(output, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    # return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))

    return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))


def best_word(s, d, l, silent=False):
    a = get_suggestions(s, d, l, silent)
    if len(a)==0:
        return (None, (None, None))
    else: 
        return a[0]

<div class="alert alert-danger">
  <strong>Run the cell below only once to build the dictionary.</strong>
</div>

In [ ]:
%%time
d, lwl = parallel_create_dictionary("testdata/big.txt")

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 52 ms, sys: 13 ms, total: 65 ms
Wall time: 54.1 s
</code></pre>

<div class="alert alert-success">
  <strong>Enter word to correct below.</strong>
</div>

In [ ]:
%%time
get_suggestions("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 56.6 ms, sys: 14.8 ms, total: 71.4 ms
Wall time: 53.7 s
Out[12]:
[(u'there', (2972, 0)),
 (u'these', (1231, 1)),
 (u'where', (977, 1)),
 (u'here', (691, 1)),
 (u'three', (584, 1)),
 (u'thee', (26, 1)),
 (u'chere', (9, 1)),
 (u'theme', (8, 1)),
 (u'the', (80030, 2)), ...
</code></pre>

In [ ]:
%%time
get_suggestions("zzffttt", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 0
  edit distance for deletions: 3
CPU times: user 57.7 ms, sys: 15.6 ms, total: 73.4 ms
Wall time: 57.3 s
Out[4]:
[]
</code></pre>

In [ ]:
%%time
best_word("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 80.6 ms, sys: 19.3 ms, total: 99.8 ms
Wall time: 59.7 s
Out[4]:
(u'there', (2972, 0))
</code></pre>

----

### 4. Optimized SPARK version: ALSO FAST

<div class="alert alert-danger">
  <strong>To run this program, restart notebook, and start executing the cells of this section starting here.</strong> <br><p>
  The difference between this version and #3 is that it does not keep the corpus in RDD form; the corpus is broadcast to the workers by the driver and used as a dictionary upon which lookups are performed. When scaling, the dictionary itself, while large, can be smaller than the documents being checked and should fit in memory (since the serial version assumes that the dictionary can be stored in memory anyway).
</div>

<div class="alert alert-info">
corresponds to `spark_3.py`
</div>

In [ ]:
'''
v 3.0 last revised 26 Nov 2015

This program is a Spark (PySpark) version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

Future improvements may entail allowing for less verbose options, 
including the output of a single recommended correction. Note also that we
have generally kept to the form of the original program, and have not
introduced any major optimizations or structural changes in this PySpark port.

To execute program:
1. Ensure "big.txt" is in the current working directory. This is the corpus
   from which the dictionary for the spellchecker will be built.
2. Check recommended single word corrections by executing get_suggestions("word") 
    in the corresponding marked box below.

Note: we did not implement entire document checking given speed of program,
      since we are parallelizing the processing of deletions of the input word
      (however, see later Spark version).

################

Example input/output:

################

get_suggestions("there")

number of possible corrections: 604
  edit distance for deletions: 3
  
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...


'''

import findspark
import os
findspark.init('/Users/K-Lo/spark-1.5.0')

from pyspark import SparkContext
sc = SparkContext()

import re

n_partitions = 6  # number of partitions to be used
max_edit_distance = 3

# helper functions
def get_n_deletes_list(w, n):
    '''given a word, derive list of strings with up to n characters deleted'''
    # since this list is generally of the same magnitude as the number of 
    # characters in a word, it may not make sense to parallelize this
    # so we use python to create the list
    deletes = []
    queue = [w]
    for d in range(n):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes
    
def copartitioned(RDD1, RDD2):
    '''check if two RDDs are copartitioned'''
    return RDD1.partitioner == RDD2.partitioner

def combine_joined_lists(tup):
    '''takes as input a tuple in the form (a, b) where each of a, b may be None (but not both) or a list
       and returns a concatenated list of unique elements'''
    concat_list = []
    if tup[1] is None:
        concat_list = tup[0]
    elif tup[0] is None:
        concat_list = tup[1]
    else:
        concat_list = tup[0] + tup[1]
        
    return list(set(concat_list))

def parallel_create_dictionary(fname):
    '''Create dictionary using Spark RDDs.'''
    # we generate and count all words for the corpus,
    # then add deletes to the dictionary
    # this is a slightly different approach from the SymSpell algorithm
    # that may be more appropriate for Spark processing
    
    print "Creating dictionary..." 
    
    ############
    #
    # process corpus
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # convert file into one long sequence of words
    make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
    replace_nonalphs = make_all_lower.map(lambda line: regex.sub(' ', line))
    all_words = replace_nonalphs.flatMap(lambda line: line.split())

    # create core corpus dictionary (i.e. only words appearing in file, no "deletes") and cache it
    # output RDD of unique_words_with_count: [(word1, count1), (word2, count2), (word3, count3)...]
    count_once = all_words.map(lambda word: (word, 1))
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()
    
    # output stats on core corpus
    print "total words processed: %i" % unique_words_with_count.map(lambda (k, v): v).reduce(lambda a, b: a + b)
    print "total unique words in corpus: %i" % unique_words_with_count.count()
    
    ############
    #
    # generate deletes list
    #
    ############
    
    # generate list of n-deletes from words in a corpus of the form: [(word1, count1), (word2, count2), ...]
     
    assert max_edit_distance>0  
    
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
                                                   (parent, get_n_deletes_list(parent, max_edit_distance)))
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
   
    ############
    #
    # combine delete elements with main dictionary
    #
    ############
    
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))
    combine = swap.union(corpus)  # combine deletes with main dictionary, eliminate duplicates
    
    ## since the dictionary will only be a lookup table once created, we can
    ## pass on as a Python dictionary rather than RDD by reducing locally and
    ## avoiding an extra shuffle from reduceByKey
    new_dict = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1]))
    
    print "total items in dictionary (corpus words and deletions): %i" % len(new_dict)
    print "  edit distance for deletions: %i" % max_edit_distance
    longest_word_length = unique_words_with_count.map(lambda (k, v): len(k)).reduce(max)
    print "  length of longest word in corpus: %i" % longest_word_length

    return new_dict, longest_word_length 

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance (an integer) between sequences.

    This code has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def get_suggestions(s, masterdict, longest_word_length=float('inf'), silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word.
    
    s: input string
    masterdict: the main dictionary (python dict), which includes deletes
             entries, is in the form of: {word: ([suggested corrections], 
                                                 frequency of word in corpus), ...}
    longest_word_length: optional identifier of longest real word in masterdict
    silent: verbose output (when False)
    '''

    if (len(s) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []

    ##########
    #
    # initialize suggestions RDD
    # suggestRDD entries: (word, (frequency of word in corpus, edit distance))
    #
    ##########
    
    ## since the dictionary will only be a lookup table once created, we can
    ## keep as Python dictionary rather than converting to RDD 
    ##### try broadcasting to workers for efficiency
    masterdict2 = sc.broadcast(masterdict)
    
    if not silent:
        print "looking up suggestions based on input word..."
    
    got_suggestions = False  # initialize flag
    init_sugg = []
    
    # check if input word is in dictionary, and is a word from the corpus (edit distance = 0)
    # if so, add input word itself and suggestions to suggestRDD
    
    if s in masterdict2.value:
        got_suggestions = True
        # dictionary values are in the form of ([suggestions], freq)
        if masterdict2.value[s][1]>0:  # frequency>0 -> real corpus word
            init_sugg = [(s, (masterdict2.value[s][1], 0))]

        # the suggested corrections for the item in dictionary (whether or not
        # the input string s itself is a valid word or merely a delete) can be 
        # valid corrections  -- essentially we serialize this portion since
        # the list of corrections tends to be very short
        
        add_sugg = [(sugg, (masterdict2.value[sugg][1], len(sugg)-len(s))) 
                        for sugg in masterdict2.value[s][0]]
        
        suggestRDD = sc.parallelize(init_sugg + add_sugg, n_partitions).cache()
        
    ##########
    #
    # process deletes on the input string 
    #
    ##########
     
    assert max_edit_distance>0
    
    list_deletes_of_s = get_n_deletes_list(s, max_edit_distance)  # this list is short
    
    add_sugg_2 = [(sugg, (masterdict2.value[sugg][1], len(s)-len(sugg))) 
                      for sugg in list_deletes_of_s if ((sugg in masterdict2.value) and
                                                        (masterdict2.value[sugg][1]>0))]
    
    add_sugg_2_RDD = sc.parallelize(add_sugg_2, n_partitions)
    
    if got_suggestions:
        suggestRDD = suggestRDD.union(add_sugg_2_RDD).cache()
    else:
        got_suggestions = True
        suggestRDD = add_sugg_2_RDD.cache()
        
    # check each item of suggestion list of all new-found suggestions 
    # the suggested corrections for the item in dictionary (whether or not
    # the delete itself is a valid word or merely a delete) can be valid corrections   
    # expand lists of list
    
    sugg_lists = [masterdict2.value[sugg][0] for sugg in list_deletes_of_s 
                      if sugg in masterdict2.value]
    sugg_lists_RDD = sc.parallelize(sugg_lists)
    list_sl = sugg_lists_RDD.flatMap(lambda x: x).map(lambda w: (w, 0))
    combine_del = list_sl.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()

    # need to recalculate actual Deverau Levenshtein distance to be within max_edit_distance for all deletes
    calc_dist = combine_del.map(lambda (w, _): (w, dameraulevenshtein(s, w)),
                                       preservesPartitioning=True)
    filter_by_dist = calc_dist.filter(lambda (w, dist): dist<=max_edit_distance)
    
    # get frequencies from main dictionary and add to suggestions list
    get_freq = filter_by_dist.filter(lambda (w, dist): w in masterdict2.value)
    del_parent_sugg = get_freq.map(lambda (w, dist): (w, (masterdict2.value[w][1], dist)),
                                     preservesPartitioning=True)
    
    if got_suggestions:
        if len(sugg_lists)>0:
            suggestRDD = suggestRDD.union(del_parent_sugg).distinct().cache()
        else:
            # no additional suggestions, leave suggest RDD intact
            pass
        
        # convert unicode to string (currently input not in unicode)
        suggestRDD = suggestRDD.map(lambda (w, _): (str(w), _)).cache()
    else:
        suggestRDD = sc.EmptyRDD() 
                   
    if not silent:
        print "number of possible corrections: %i" % suggestRDD.count()
        print "  edit distance for deletions: %i" % max_edit_distance

    ##########
    #
    # optionally, sort RDD for output
    #
    ##########
    
    # suggest_RDD is in the form: [(word, (freq, editdist)), (word, (freq, editdist)), ...]
    # there does not seem to be a straightforward way to sort by both primary and secondary keys in Spark
    # this is a documented issue: one option is to simply work with a list since there are likely not
    # going to be an extremely large number of recommended suggestions
    
    output = suggestRDD.collect()
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(output, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    # return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))

    if len(output)>0:
        return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))
    else:
        return []

def best_word(s, d, l, silent=False):
    a = get_suggestions(s, d, l, silent)
    if len(a)==0:
        return (None, (None, None))
    else: 
        return a[0]

<div class="alert alert-danger">
  <strong>Run the cell below only once to build the dictionary.</strong>
</div>

In [ ]:
%%time
d, lwl = parallel_create_dictionary("testdata/big.txt")

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 10.9 s, sys: 1.01 s, total: 11.9 s
Wall time: 42.1 s
</code></pre>

<div class="alert alert-success">
  <strong>Enter word to correct below.</strong>
</div>

In [ ]:
%%time
get_suggestions("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 25.3 s, sys: 1.65 s, total: 26.9 s
Wall time: 1min 3s
Out[6]:
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...
</code></pre>

In [ ]:
%%time
get_suggestions("zzffttt", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 0
  edit distance for deletions: 3
CPU times: user 15.9 s, sys: 1.3 s, total: 17.2 s
Wall time: 18.5 s
Out[6]:
[]
</code></pre>

In [ ]:
%%time
best_word("there", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 23.3 s, sys: 1.3 s, total: 24.6 s
Wall time: 1min 2s
Out[4]:
('there', (2972, 0))
</code></pre>

### 5. SPARK version document check

<div class="alert alert-danger">
  <strong>To run this program, restart notebook, and start executing the cells of this section starting here.</strong> <br><p>
  This version parallelizes the word check for all the words in a document, using word-level correction. Since SPARK does not permit RDD manipulation from within an RDD transformation (i.e. no parallelism within a parallel task), we converted the `get_suggestions` function that acts on an individual word to a serial method. This allows us to then parallelize across multiple words in a document. <i>This is a reasonable trade off when the number of words in a document is much larger compared to the number of suggestions that will likely be found for any given word)</i>. <br><p>
  <b>IMPORTANT NOTE REGARDING PERFORMANCE RESULTS:</b> The (modified) `no_RDD_get_suggestions` function still returns an entire list of all possible suggestions to the calling function (e.g. for context checking), even if only the top match is used or required. Future improvements may be made to `no_RDD_get_suggestions` to terminate early once a "top" match (e.g. minimum edit distance) is found; a speedup in that function will in turn lead to a performance improvement of the document checking function as well.
</div>

<div class="alert alert-info">
corresponds to `spark_4.py`
</div>

In [ ]:
'''
v 4.0 last revised 27 Nov 2015

This program is a Spark (PySpark) version of a spellchecker based on SymSpell, 
a Symmetric Delete spelling correction algorithm developed by Wolf Garbe 
and originally written in C#.

From the original SymSpell documentation:

"The Symmetric Delete spelling correction algorithm reduces the complexity 
 of edit candidate generation and dictionary lookup for a given Damerau-
 Levenshtein distance. It is six orders of magnitude faster and language 
 independent. Opposite to other algorithms only deletes are required, 
 no transposes + replaces + inserts. Transposes + replaces + inserts of the 
 input term are transformed into deletes of the dictionary term.
 Replaces and inserts are expensive and language dependent: 
 e.g. Chinese has 70,000 Unicode Han characters!"

For further information on SymSpell, please consult the original documentation:
  URL: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
  Description: blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/

The current version of this program will output all possible suggestions for
corrections up to an edit distance (configurable) of max_edit_distance = 3. 

To execute program:
1. Ensure "big.txt" is in the current working directory. This is the corpus
   from which the dictionary for the spellchecker will be built.
2a. Check recommended single word corrections by executing no_RDD_get_suggestions("word") 
    in the corresponding marked box below; or
2b. Check single word corrections for a document by executing
    correct_document("<file>") in the corresponding marked box below.

################

Example input/output:

################

no_RDD_get_suggestions("there")

number of possible corrections: 604
  edit distance for deletions: 3
  
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...

####

correct_document("OCRsample.txt")

Finding misspelled words in your document...
    Unknown words (line number, word in text):
[(11, 'oonipiittee'), (42, 'senbrnrgs'), (82, 'ghmhvestigat')]
    Words with suggested corrections (line number, word in text, top match):
[(3, 'taiths --> faith'), (13, 'gjpt --> get'), (13, 'tj --> to'), 
 (13, 'mnnff --> snuff'), (15, 'bh --> by'), (15, 'uth --> th'), ...
 (15, 'unuer --> under'),

---
total words checked: 700
total unknown words: 3
total potential errors found: 94

'''

import findspark
import os
findspark.init('/Users/K-Lo/spark-1.5.0')

from pyspark import SparkContext
sc = SparkContext()

import re

n_partitions = 6  # number of partitions to be used
max_edit_distance = 3

# helper functions
def get_n_deletes_list(w, n):
    '''given a word, derive list of strings with up to n characters deleted'''
    # since this list is generally of the same magnitude as the number of 
    # characters in a word, it may not make sense to parallelize this
    # so we use python to create the list
    deletes = []
    queue = [w]
    for d in range(n):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes
    
def copartitioned(RDD1, RDD2):
    '''check if two RDDs are copartitioned'''
    return RDD1.partitioner == RDD2.partitioner

def combine_joined_lists(tup):
    '''takes as input a tuple in the form (a, b) where each of a, b may be None (but not both) or a list
       and returns a concatenated list of unique elements'''
    concat_list = []
    if tup[1] is None:
        concat_list = tup[0]
    elif tup[0] is None:
        concat_list = tup[1]
    else:
        concat_list = tup[0] + tup[1]
        
    return list(set(concat_list))

def parallel_create_dictionary(fname):
    '''Create dictionary using Spark RDDs.'''
    # we generate and count all words for the corpus,
    # then add deletes to the dictionary
    # this is a slightly different approach from the SymSpell algorithm
    # that may be more appropriate for Spark processing
    
    print "Creating dictionary..." 
    
    ############
    #
    # process corpus
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # convert file into one long sequence of words
    make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
    replace_nonalphs = make_all_lower.map(lambda line: regex.sub(' ', line))
    all_words = replace_nonalphs.flatMap(lambda line: line.split())

    # create core corpus dictionary (i.e. only words appearing in file, no "deletes") and cache it
    # output RDD of unique_words_with_count: [(word1, count1), (word2, count2), (word3, count3)...]
    count_once = all_words.map(lambda word: (word, 1))
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()
    
    # output stats on core corpus
    print "total words processed: %i" % unique_words_with_count.map(lambda (k, v): v).reduce(lambda a, b: a + b)
    print "total unique words in corpus: %i" % unique_words_with_count.count()
    
    ############
    #
    # generate deletes list
    #
    ############
    
    # generate list of n-deletes from words in a corpus of the form: [(word1, count1), (word2, count2), ...]
     
    assert max_edit_distance>0  
    
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
                                                   (parent, get_n_deletes_list(parent, max_edit_distance)))
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
   
    ############
    #
    # combine delete elements with main dictionary
    #
    ############
    
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))
    combine = swap.union(corpus)  # combine deletes with main dictionary, eliminate duplicates
    
    ## since the dictionary will only be a lookup table once created, we can
    ## pass on as a Python dictionary rather than RDD by reducing locally and
    ## avoiding an extra shuffle from reduceByKey
    new_dict = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1]))
    
    print "total items in dictionary (corpus words and deletions): %i" % len(new_dict)
    print "  edit distance for deletions: %i" % max_edit_distance
    longest_word_length = unique_words_with_count.map(lambda (k, v): len(k)).reduce(max)
    print "  length of longest word in corpus: %i" % longest_word_length

    return new_dict, longest_word_length    

def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance (an integer) between sequences.

    This code has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

def no_RDD_get_suggestions(s, masterdict, longest_word_length=float('inf'), silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word.
    
    Note: serialized version for Spark document correction.
    
    s: input string
    masterdict: the main dictionary (python dict), which includes deletes
             entries, is in the form of: {word: ([suggested corrections], 
                                                 frequency of word in corpus), ...}
    longest_word_length: optional identifier of longest real word in masterdict
    silent: verbose output (when False)
    '''

    if (len(s) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []

    ##########
    #
    # initialize suggestions list
    # suggestList entries: (word, (frequency of word in corpus, edit distance))
    #
    ##########
    
    if not silent:
        print "looking up suggestions based on input word..."
    
    suggestList = []
    
    # check if input word is in dictionary, and is a word from the corpus (edit distance = 0)
    # if so, add input word itself and suggestions to suggestRDD
    
    if s in masterdict:
        init_sugg = []
        # dictionary values are in the form of ([suggestions], freq)
        if masterdict[s][1]>0:  # frequency>0 -> real corpus word
            init_sugg = [(str(s), (masterdict[s][1], 0))]

        # the suggested corrections for the item in dictionary (whether or not
        # the input string s itself is a valid word or merely a delete) can be 
        # valid corrections  -- essentially we serialize this portion since
        # the list of corrections tends to be very short
        
        add_sugg = [(str(sugg), (masterdict[sugg][1], len(sugg)-len(s))) 
                        for sugg in masterdict[s][0]]
        
        suggestList = init_sugg + add_sugg
        
    ##########
    #
    # process deletes on the input string 
    #
    ##########
     
    assert max_edit_distance>0
    
    list_deletes_of_s = get_n_deletes_list(s, max_edit_distance)  # this list is short
    
    # check suggestions is in dictionary and is a real word
    add_sugg_2 = [(str(sugg), (masterdict[sugg][1], len(s)-len(sugg))) 
                      for sugg in list_deletes_of_s if ((sugg in masterdict) and
                                                        (masterdict[sugg][1]>0))]
    
    suggestList += add_sugg_2
        
    # check each item of suggestion list of all new-found suggestions 
    # the suggested corrections for any item in dictionary (whether or not
    # the delete itself is a valid word or merely a delete) can be valid corrections   
    # expand lists of list
    
    sugg_lists = [masterdict[sugg][0] for sugg in list_deletes_of_s if sugg in masterdict]
    list_sl = [(val, 0) for sublist in sugg_lists for val in sublist]
    combine_del = list(set((list_sl))) 

    # need to recalculate actual Deverau Levenshtein distance to be within 
    # max_edit_distance for all deletes; also check that suggestion is a real word
    filter_by_dist = []
    for item in combine_del:
        calc_dist = dameraulevenshtein(s, item[0])
        if (calc_dist<=max_edit_distance) and (item[0] in masterdict):
            filter_by_dist += [(item[0], calc_dist)]
        
    # get frequencies from main dictionary and add new suggestions to suggestions list
    suggestList += [(str(item[0]), (masterdict[item[0]][1], item[1]))
                            for item in filter_by_dist]
    
    output = list(set(suggestList))
    
    if not silent:
        print "number of possible corrections: %i" % len(output)
        print "  edit distance for deletions: %i" % max_edit_distance

    ##########
    #
    # optionally, sort RDD for output
    #
    ##########
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(output, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    # return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))

    if len(output)>0:
        return sorted(output, key = lambda (term, (freq, dist)): (dist, -freq))
    else:
        return []
    
def correct_document(fname, d, lwl=float('inf'), printlist=True):
    '''Correct an entire document using word-level correction.
    
    Note: Uses a serialized version of an individual word checker. 
    
    fname: filename
    d: the main dictionary (python dict), which includes deletes
             entries, is in the form of: {word: ([suggested corrections], 
                                                 frequency of word in corpus), ...}
    lwl: optional identifier of longest real word in masterdict
    printlist: identify unknown words and words with error (default is True)
    '''
    
    # broadcast lookup dictionary to workers
    bd = sc.broadcast(d)
    
    print "Finding misspelled words in your document..." 
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # convert file into one long sequence of words with the line index for reference
    make_all_lower = sc.textFile(fname).map(lambda line: line.lower()).zipWithIndex()
    replace_nonalphs = make_all_lower.map(lambda (line, index): (regex.sub(' ', line), index))
    flattened = replace_nonalphs.map(lambda (line, index): 
                                 [(i, index) for i in line.split()]).flatMap(list)
    
    # create RDD with (each word in document, corresponding line index) 
    # key value pairs and cache it
    all_words = flattened.partitionBy(n_partitions).cache()
    
    # check all words in parallel --  stores whole list of suggestions for each word
    get_corrections = all_words.map(lambda (w, index): 
                                    (w, (no_RDD_get_suggestions(w, bd.value, lwl, True), index)),
                                     preservesPartitioning=True).cache()
    
    # UNKNOWN words are words where the suggestion list is empty
    unknown_words = get_corrections.filter(lambda (w, (sl, index)): len(sl)==0)
    if printlist:
        print "    Unknown words (line number, word in text):"
        print unknown_words.map(lambda (w, (sl, index)): (index, str(w))).sortByKey().collect()
    
    # ERROR words are words where the word does not match the first tuple's word (top match)
    error_words = get_corrections.filter(lambda (w, (sl, index)): len(sl)>0 and w!=sl[0][0]) 
    if printlist:
        print "    Words with suggested corrections (line number, word in text, top match):"
        print error_words.map(lambda (w, (sl, index)): 
                                 (index, str(w) + " --> " +
                                         str(sl[0][0]))).sortByKey().collect()
    
    gc = sc.accumulator(0)
    get_corrections.foreach(lambda x: gc.add(1))
    uc = sc.accumulator(0)
    unknown_words.foreach(lambda x: uc.add(1))
    ew = sc.accumulator(0)
    error_words.foreach(lambda x: ew.add(1))
    
    print "-----"
    print "total words checked: %i" % gc.value
    print "total unknown words: %i" % uc.value
    print "total potential errors found: %i" % ew.value

    return

<div class="alert alert-danger">
  <strong>Run the cell below only once to build the dictionary.</strong>
</div>

In [ ]:
%%time
d, lwl = parallel_create_dictionary("testdata/big.txt")

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 10.9 s, sys: 976 ms, total: 11.8 s
Wall time: 41.2 s
</code></pre>

<div class="alert alert-success">
  <strong>Enter word to correct below.</strong>
</div>

In [ ]:
%%time
no_RDD_get_suggestions("theref", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 604
  edit distance for deletions: 3
CPU times: user 56.3 ms, sys: 4.17 ms, total: 60.5 ms
Wall time: 58.2 ms
Out[3]:
[('there', (2972, 0)),
 ('these', (1231, 1)),
 ('where', (977, 1)),
 ('here', (691, 1)),
 ('three', (584, 1)),
 ('thee', (26, 1)),
 ('chere', (9, 1)),
 ('theme', (8, 1)),
 ('the', (80030, 2)), ...
</code></pre>

In [ ]:
%%time
no_RDD_get_suggestions("zzffttt", d, lwl)

<pre><code>
looking up suggestions based on input word...
number of possible corrections: 0
  edit distance for deletions: 3
CPU times: user 419 µs, sys: 108 µs, total: 527 µs
Wall time: 435 µs
Out[3]:
[]
</code></pre>

<div class="alert alert-success">
  <strong>Enter file name of document to correct below.</strong>
</div>

In [ ]:
%%time
correct_document("testdata/OCRsample.txt", d, lwl)

<pre><code>
Finding misspelled words in your document...
    Unknown words (line number, word in text):
[(11, 'oonipiittee'), (42, 'senbrnrgs'), (82, 'ghmhvestigat')]
    Words with suggested corrections (line number, word in text, top match):
[(3, 'taiths --> faith'), (13, 'gjpt --> get'), (13, 'tj --> to'), (13, 'mnnff --> snuff'), (15, 'bh --> by'), (15, 'uth --> th'), (15, 'unuer --> under'), (15, 'snc --> sac'), (20, 'mthiitt --> thirty'), (21, 'cas --> was'), (22, 'pythian --> scythian'), (26, 'brainin --> brain'), (27, 'jfl --> of'), (28, 'eug --> dug'), (28, 'stice --> stick'), (28, 'blaci --> black'), (28, 'ji --> i'), (28, 'debbs --> debts'), (29, 'nericans --> americans'), (30, 'ergs --> eggs'), (30, 'ainin --> again'), (31, 'trumped --> trumpet'), (32, 'erican --> american'), (33, 'thg --> the'), (33, 'nenance --> penance'), (33, 'unorthodox --> orthodox'), (34, 'rgs --> rags'), (34, 'sln --> son'), (38, 'eu --> e'), (38, 'williaij --> william'), (40, 'fcsf --> ff'), (40, 'ber --> be'), (42, 'thpt --> that'), (42, 'unorthodoxy --> orthodox'), (44, 'fascism --> fascia'), (62, 'loo --> look'), (65, 'ththn --> then'), (65, 'thl --> the'), (65, 'yktcn --> skin'), (65, 'scbell --> bell'), (65, 'ife --> if'), (66, 'thi --> the'), (68, 'saij --> said'), (69, 'cornr --> corner'), (69, 'defendants --> defendant'), (69, 'nists --> lists'), (72, 'ro --> to'), (74, 'ath --> at'), (75, 'rg --> re'), (75, 'acrific --> pacific'), (75, 'tti --> tit'), (77, 'korea --> more'), (78, 'doatli --> death'), (78, 'ro --> to'), (81, 'ry --> by'), (81, 'ith --> it'), (81, 'kl --> ll'), (81, 'ech --> each'), (82, 'rg --> re'), (82, 'rb --> re'), (82, 'nb --> no'), (83, 'rosenbt --> rodent'), (83, 'rgs --> rags'), (84, 'coriritted --> committed'), (86, 'fighti --> fight'), (88, 'bths --> baths'), (88, 'tchf --> the'), (91, 'ro --> to'), (91, 'ijb --> in'), (92, 'telegrnm --> telegram'), (92, 'rson --> son'), (92, 'jillia --> william'), (92, 'patt --> part'), (93, 'ecretdry --> secretary'), (95, 'purview --> purves'), (95, 'rder --> order'), (99, 'gor --> for'), (99, 'rg --> re'), (99, 'enb --> end'), (99, 'dthethg --> teeth'), (99, 'ro --> to'), (99, 'ared --> are'), (100, 'dri --> dry'), (100, 'yfu --> you'), (100, 'vthnz --> the'), (100, 'sacc --> sac'), (101, 'rosi --> rose'), (101, 'rg --> re'), (101, 'ile --> ill'), (102, 'jhy --> why'), (102, 'fnir --> fair'), (102, 'azi --> ai'), (103, 'fascist --> fascia'), (104, 'nb --> no')]
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 9.9 s, sys: 619 ms, total: 10.5 s
Wall time: 1min 3s
</code></pre>

In [ ]:
%%time
correct_document("testdata/OCRsample.txt", d, lwl, False)

<pre><code>
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 9.33 s, sys: 505 ms, total: 9.83 s
Wall time: 56.3 s
</code></pre>

<pre><code>
# additional tests (local machine)

Finding misspelled words in your document...
-----
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
266.66 seconds to run
-----
</code></pre>

<pre><code>
Finding misspelled words in your document...
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
2897.28 seconds to run
</code></pre>

# AWS Experiments

<div class="alert alert-info">
It took a long time to debug problems with AWS, but in the end the programs ran. The key thing I learned is that when broadcasting the dictionary to the workers, we have to set the executor memory (e.g. to 5G or 8G) explicitly to ensure that the workers can store it (was originally left out). We also include sample output for the document-level Spark version below.
</div>

## individual word checks

<pre><core>
# 2 executors, 4 cores, 16 partitions
# "there"

serial: 62 ms
spark 1: 96.31 s
spark 2: 22.62 s
spark 3: 41.35 s
</core></pre>

<div class="alert alert-info">
This result was interesting, as the `reduceByKeyLocally` performs much worse with many workers. This is the reverse results compared to those obtained when running the Spark versions 2 and 3 on my local machine.  The cost of a potential shuffle between building the dictionary and the word check seems negligble now, compared to the overhead of setting up communication with all the workers and then having to bring everything back to the driver.
</div>

## document checks (on AWS)

<pre><core>

# Serial version:

total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
19.01 seconds to run
-----

total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
384.75 seconds to run
-----

-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
4305.46 seconds to run
-----
</core></pre>

<div class="alert alert-info">
The serial version now lags behind the Spark versions once the documents size starts getting very big.
</div>

### 2 executors, 4 cores, 16 partitions

<pre><core>
Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
-----
35.37 seconds to run
-----
 
Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
41.82 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
125.45 seconds to run
-----

-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
995.66 seconds to run
-----
</core></pre>

<div class="alert alert-info">
The Spark versions begin to show their strength at the document sizes get larger. We began to fiddle with the various parameters (e.g. number of executors, number of cores, number of partitions set in the code).  Note that we are still generating all suggestions for EACH words - for the purposes of comparing performance this is fine, but we would likely have to optimize the code (in the manner of the final serial version where we can terminate searches early once suggestions of the lowest edit distances are found) if we were to run this on even larger documents.
</div>

### 2 executors, 4 cores, 32 partitions

<pre><core>
Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
-----
34.84 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
66.17 seconds to run


Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
150.06 seconds to run
-----

*** when I left memory at 5G
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
2357.24 seconds to run
-----

*** boosted memory to 8G
Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
972.82 seconds to run
-----

*** changed reduceByKeyLocally to reduceByKey + collectAsMap in createDocument
Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
992.03 seconds to run
-----
</core></pre>

### 4 executors, 4 cores, 16 partitions

<pre><core>

Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
-----
66.97 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
148.86 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
240.04 seconds to run
-----
</core></pre>

### 4 executors, 4 cores, 32 partitions

<pre><core>
Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
-----
71.95 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
163.45 seconds to run


Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
246.38 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
1191.01 seconds to run
</core></pre>

### 4 executors, 4 cores, 64 partitions

<pre><core>

Please wait...
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
-----
69.25 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
-----
198.92 seconds to run
-----

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 12029
total unknown words: 19
total potential errors found: 719
-----
286.15 seconds to run

Document correction... Please wait...
-------------------------------------
finding corrections for document
Finding misspelled words in your document...
-----
total words checked: 131340
total unknown words: 325
total potential errors found: 8460
-----
1283.60 seconds to run
-----
</core></pre>

Future investigations may entail exploring and tuning these parameters, and experimenting with others, to optimize performance.

# Appendix

<div class="alert alert-info">
  <strong>In this section, we document various attempts at optimizing the original Spark (v 1.0) program.</strong> <br><p>
  Note we also examined the output at localhost:4040 to identify the time-consuming stages.
</div>

### building dictionary

<div class="alert alert-success">
  <strong>Removed setting of executor memory decreased time to build dictionary.</strong> <br><p>
  Replace:
  <pre><code>  
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
  </code></pre><p>
  with:
  <pre><code>  
  from pyspark import SparkContext
  sc = SparkContext()
  </code></pre>
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 113 ms, sys: 24.6 ms, total: 138 ms
Wall time: 2min 48s
</code></pre>

<div class="alert alert-danger">
  <strong>Setting `minPartitions` argument in `textFile` did not improve performance.</strong> <br><p>
  Replace:
  <pre><code>  
make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
  </code></pre><p>
  with:
  <pre><code>  
make_all_lower = sc.textFile(fname, minPartitions = n_partitions).map(lambda line: line.lower())
  </code></pre>
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 133 ms, sys: 28.9 ms, total: 162 ms
Wall time: 2min 55s
</code></pre>

<div class="alert alert-danger">
  <strong>Repartitioning and sorting corpus after processing core words did not improve performance.</strong> <br><p>
  Replace:
  <pre><code>  
unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions = n_partitions).cache()
  </code></pre><p>
  with:
  <pre><code>  
unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b).repartitionAndSortWithinPartitions(numPartitions =n_partitions).cache()
  </code></pre>
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 150 ms, sys: 31.8 ms, total: 182 ms
Wall time: 3min 5s
</code></pre>

<div class="alert alert-danger">
  <strong>Explicitly setting use_unicode to False, which is documented as possibly leading to a faster read, did not significantly improve performance.</strong> <br><p>
  Replace:
  <pre><code>  
make_all_lower = sc.textFile(fname).map(lambda line: line.lower())
  </code></pre><p>
  with:
  <pre><code>  
make_all_lower = sc.textFile(fname, use_unicode=False).map(lambda line: line.lower())
  </code></pre>
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 106 ms, sys: 24.3 ms, total: 130 ms
Wall time: 2min 46s
</code></pre>

<div class="alert alert-success">
  <strong>Removing parallelism when generating individual 1-deletes, 2-deletes, and 3-deletes improved performance. </strong> This is likely due to the fact that these lists are pretty short relative to the length of the corpus; parallelizing them introduces overhead.<br><p>
  Replaced `get_deletes_list` with `get_n_deletes_list` function, and modified `parallel_create_dictionary` accordingly. <br><p> <b>See Spark v. 2.0 above.</b> 
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 52 ms, sys: 13 ms, total: 65 ms
Wall time: 54.1 s
</code></pre>

<div class="alert alert-success">
  <strong>Replacing a `join` with `union`, and `reduceByKey` with `reduceByKeyLocally` to return dictionary to driver without an extra shuffle, provides comparable performance  </strong> (however, dictionary is now a python dictionary, and not an RDD).<br><p>
 Modified `parallel_create_dictionary` accordingly. <br><p> <b>See Spark v. 3.0 above.</b> 
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 10.9 s, sys: 1.01 s, total: 11.9 s
Wall time: 42.1 s
</code></pre>

<div class="alert alert-danger">
  <strong>Replacing `reduceByKeyLocally` with a `reduceByKey` and `collectAsMap` did not significantly improve performance.</strong> This is because an extra shuffle was introduced. <br><p>
  Replace:
  <pre><code>  
new_dict = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1]))
  </code></pre><p>
  with:
  <pre><code>  
new_dict = combine.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])).collectAsMap()
  </code></pre>
</div>

<pre><code> 
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 7.16 s, sys: 921 ms, total: 8.08 s
Wall time: 1min 7s
</code></pre>

<div class="alert alert-danger">
  <strong>Modifying the initial `reduceByKey` when reading the corpus to remove explicit partitioning (now not needed with the use of `union`) did not significantly improve performance.</strong> <br><p>
  Replace:
  <pre><code>  
unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, numPartitions=n_partitions).cache()
  </code></pre><p>
  with:
  <pre><code>  
unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b).cache()
  </code></pre>
</div>

<pre><code> 
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 9.05 s, sys: 824 ms, total: 9.87 s
Wall time: 43.4 s
</code></pre>

<div class="alert alert-danger">
  <strong>Increasing number of partitions from 6 to 8, on our local machine (dual core, 2015 MacBook Pro) did not significantly improve performance.</strong> <br><p>
  Replace:
  <pre><code>  
n_partitions = 6 # number of partitions to be used
  </code></pre><p>
  with:
  <pre><code>  
n_partitions = 8 # number of partitions to be used
  </code></pre>
</div>

<pre><code>
Creating dictionary...
total words processed: 1105285
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 11.6 s, sys: 1.08 s, total: 12.7 s
Wall time: 45.5 s
</code></pre>

### implement document checking

<div class="alert alert-success">
  <strong>Serializing `get_suggestions` allowed us to parallelize the checking of words within an entire document. </strong> Spark does not appear to permit parallelizing a task within a parallel task (transformation). <br><p>
  Replaced `get_suggestions` with `no_RDD_get_suggestions` function, and modified `correct_document` accordingly. 
</div>

<pre><code> 
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 1min 6s, sys: 1.36 s, total: 1min 8s
Wall time: 2min 1s
</code></pre>

<div class="alert alert-success">
  <strong>Broadcasting dictionary to workers before checking document improved performance.</strong> <br><p> 
  Added:
  <pre><code>  
    # broadcast lookup dictionary to workers
    bd = sc.broadcast(d)
  </code></pre><p>
  Modified `get_corrections` to:
  <pre><code>  
    get_corrections = all_words.map(lambda (w, index): (w, (get_suggestions(w, bd.value, lwl, True), index)), preservesPartitioning=True).cache()
  </code></pre>
</div>

<pre><code>
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 10.6 s, sys: 620 ms, total: 11.2 s
Wall time: 57.3 s
</code></pre>

<div class="alert alert-success">
  <strong>Converting counters for words in dictionary, unknown words, and error words to accumulators only improved performance nominally (may not be significant due to variations between runs).</strong><br><p> <b>These changes have been implemented. See Spark v. 4.0 above.</b> <br><p>
  Add:
  <pre><code>  
    gc = sc.accumulator(0)
    get_corrections.foreach(lambda x: gc.add(1))
    uc = sc.accumulator(0)
    unknown_words.foreach(lambda x: uc.add(1))
    ew = sc.accumulator(0)
    error_words.foreach(lambda x: ew.add(1))
  </code></pre><p>
  Modified print statements to read:
  <pre><code>  
    print "total words checked: %i" % gc.value
    print "total unknown words: %i" % uc.value
    print "total potential errors found: %i" % ew.value
  </code></pre>
</div>

<pre><code> 
Finding misspelled words in your document...
-----
total words checked: 700
total unknown words: 3
total potential errors found: 94
CPU times: user 9.33 s, sys: 505 ms, total: 9.83 s
Wall time: 56.3 s 
</code></pre>

<div class="alert alert-danger">
  <strong>We tried using the MultiProcessing (Pool) module in Python in conjunction
  with our Spark commands for the get_suggestions method.</strong> Unfortunately, this did not
  seem to work, and the output gave errors causing system instability. Occasionally, it would give key errors, while at other times, it would give an error of the type: "PythonRDD does not exist in the JVM". This instability seems to be documented: http://stackoverflow.com/questions/32443362/passing-sparkcontext-to-new-process-python-multiprocessing-module. In any event, this is something that might be subject to further investigation in the future. <br><p>
  Added:
  <pre><code>  
    def f(x):
       val, dictionary, longestword, silent = x
       return get_suggestions(val, dictionary, longestword, silent)
  </code></pre><p>
  and to `create_dictionary`:
  <pre><code>  
    # parallelization via Pool
    jobs = [(w, bd.value, lwl, False) for w in doc_words]
    suggestion_lists = p.map(f, jobs)
  </code></pre>
</div>